# preparing samples from the model for alpaca farm

In [1]:
from datasets import load_dataset
from alpaca_farm.auto_annotations import PairwiseAutoAnnotator


In [4]:
dataset = load_dataset("json", data_files="/Users/aldo/Downloads/samples.jsonl", split="train")
#shuffle the dataset
dataset = dataset.shuffle()

In [5]:
def rename_columns_0(example):
    example["instruction"] = example["prompt"]
    example["output"] = example["answer_1"]
    # example["output"] = example["answer_2"]
    # example["input"] = ""
    example["dataset"] = ""
    example["datasplit"] = "train"
    return example
def rename_columns_1(example):
    example["instruction"] = example["prompt"]
    # example["output"] = example["answer_1"]
    example["output"] = example["answer_2"]
    # example["input"] = ""
    example["dataset"] = ""
    example["datasplit"] = "train"
    return example
dataset_0 = dataset.map(rename_columns_0, remove_columns=["prompt", "answer_1", "answer_2"])
dataset_1 = dataset.map(rename_columns_1, remove_columns=["prompt", "answer_1", "answer_2"])

Map:   0%|          | 0/116728 [00:00<?, ? examples/s]

Map:   0%|          | 0/116728 [00:00<?, ? examples/s]

In [6]:
dataset_20k_0 = dataset_0.select(range(20000))
dataset_20k_1 = dataset_1.select(range(20000))
#select last 1000 examples
dataset_1k_0 = dataset_0.select(range(20000, 21000))
dataset_1k_1 = dataset_1.select(range(20000, 21000))


In [7]:
dataset_20k_0.to_csv("/Users/aldo/Downloads/samples_20k_0.csv")
dataset_20k_1.to_csv("/Users/aldo/Downloads/samples_20k_1.csv")
dataset_1k_0.to_csv("/Users/aldo/Downloads/samples_1k_0.csv")
dataset_1k_1.to_csv("/Users/aldo/Downloads/samples_1k_1.csv")

Creating CSV from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1704011

# postprocessing after sampling

In [19]:
annotated = load_dataset("json", data_files="/Users/aldo/miniconda3/envs/rl4llm/lib/python3.10/site-packages/alpaca_eval/evaluators_configs/alpaca_eval_gpt4/annotations_seed0_configs.json", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [20]:
errors = annotated.filter(lambda x: x["preference"] not in [1,2])
print(f"there are {len(errors)} out of {len(annotated)} errors where the preference is not 1 or 2")

Filter:   0%|          | 0/20636 [00:00<?, ? examples/s]

there are 1501 out of 20636 errors where the preference is not 1 or 2


In [21]:
without_errors = annotated.filter(lambda x: x["preference"] in [1,2])

Filter:   0%|          | 0/20636 [00:00<?, ? examples/s]

In [22]:
def build_rm_format(example):
  example['prompt'] = example['instruction']
  if example['preference'] == 1:
    example['chosen'] = example['output_1']
    example['rejected'] = example['output_2']
  elif example['preference'] == 2:
    example['chosen'] = example['output_2']
    example['rejected'] = example['output_1']
  else: 
    raise ValueError("preference must be 1 or 2")
  return example

without_errors_rm = without_errors.map(build_rm_format, remove_columns=["output_1", "output_2", "preference", "instruction"])
without_errors

Map:   0%|          | 0/19135 [00:00<?, ? examples/s]

Dataset({
    features: ['output_1', 'annotator', 'price_per_example', 'preference', 'time_per_example', 'output_2', 'instruction'],
    num_rows: 19135
})

In [23]:
previous_data = load_dataset("json", data_files="/Users/aldo/Downloads/samples_rm.jsonl", split="train")

In [24]:
#get data points that are in without_errors but not in previous data
new_data = without_errors_rm.filter(lambda x: x["prompt"] not in previous_data["prompt"], num_proc=8)

Filter (num_proc=8):   0%|          | 0/19135 [00:00<?, ? examples/s]

In [25]:
new_data

Dataset({
    features: ['annotator', 'price_per_example', 'time_per_example', 'prompt', 'chosen', 'rejected'],
    num_rows: 500
})

In [26]:
import json

# with open("/Users/aldo/Downloads/samples_rm.jsonl", "w") as f:
#   for sample in without_errors_rm:
#     f.write(json.dumps(sample))
#     f.write("\n")
    

with open("/Users/aldo/Downloads/samples_rm_eval.jsonl", "w") as f:
  for sample in new_data:
    f.write(json.dumps(sample))
    f.write("\n")